# Downloading, extracting and loading the methylation data to sqlite DB

----

## <span class="label label-info">Source of data </span>
- The article : http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4107146/


--- 
## <span class="label label-warning">NOTES </span>
---


In [9]:
import pandas as pd
import numpy as np

import os
import subprocess
import time
import glob

import re

In [2]:
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/collect_data/'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

----

## Dowload data

- I downloaded the family soft file from geo http://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?token=djghdmayauuisvo&acc=GSE48684


In [6]:
%%bash
#wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE48nnn/GSE48684/soft/GSE48684_family.soft.gz
#gunzip GSE48684_family.soft.gz

---
### Preprocessing: 

This is the first time im working with soft files, so this is kinda hackish.

Extracting methylation data from the big soft file
- First write each part of the soft file to uniwue files for sample
- write the soft to the csv with the columns needed
- annotate each raw, with a sample attribute  

In [8]:
#Write all samples to different files from the one big soft file, this is kinda hacking
subprocess.call('mkdir samples',shell=True)

out_f=None
write=False
with open('GSE48684_family.soft') as in_f:
    for line in in_f:
        if (line.split()[0].strip()=='^SAMPLE'):
            write=True
            sample= line.split()[2].strip()

            if out_f!=None:
                out_f.close()
            out_f=open('samples/'+sample+'.soft','w')

        if(write):
            out_f.write(line)
out_f.close()

In [23]:
#Write files from soft to csv-a
#- missing data!
with open('met_loading_table.csv','w') as out_f:
    out_f.write('#sample\tprobe_id\tbeta\n')
    for f in glob.glob('samples/*.soft'):
        with open(f) as in_f:
            sample=os.path.basename(f).split('.')[0]
            write=False
            for line in in_f:
                if(write and line.split()[0].strip()!='!sample_table_end'):
                    out_f.write(sample+'\t'+'\t'.join(line.split('\t')[:2])+'\n')
                    
                if (line.split()[0].strip()=='ID_REF'):
                    write=True               

#### Extract the tumor-normal classification from soft files

In [25]:
samples,diseases=[],[]
for f in glob.glob('samples/*.soft'):
        with open(f) as in_f:
            samples.append(os.path.basename(f).split('.')[0])

            for line in in_f:
                if(len(re.findall('disease status',line))==1):
                    diseases.append(line.split()[-1])
                    break
    
disease_class=pd.DataFrame()
disease_class['disease']=diseases
disease_class['sample']=samples
disease_class.to_csv('disease_annot.csv',header=False,index=False,sep='\t')

#### Sqliter util function

In [26]:
#change tmpdir, or it will be full
os.environ['TMPDIR']='/nagyvinyok/adat84/sotejedlik/ribli/tmp'

#sql exetcuter func
def run_sqlilte3(command,db,output=''):
    start=time.time()
    with open('tempf.sql','w') as tempf:
        tempf.write(command)
        
    if output != '':
        output=' > '+output
    
    try:
        print subprocess.check_output('/usr/bin/sqlite3 '+ db + ' < tempf.sql '+ output,
                                      shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError, e:
        print e.output
    
    subprocess.call(['rm','tempf.sql'])
    print 'It took',int(time.time()-start),'s'

### Load data to sqlite database

In [27]:
#Load data to sqlite table
#- Its 2.2 GB so this takes a while...

run_sqlilte3("""
.separator "\t"

CREATE TABLE  meth(
sample TEXT,
probe TEXT,
beta NUMERIC);

.import met_loading_table.csv meth

--delete inserted header
DELETE FROM meth WHERE sample = 'sample';

--delete missing values
DELETE FROM meth WHERE beta = '';

""",db='../db/meth_db')


It took 318 s


In [28]:
#Load disease annotation to sqlite db

run_sqlilte3("""
.separator "\t"

CREATE TABLE  disease_annot(
disease TEXT,
sample TEXT);

.import disease_annot.csv disease_annot

""",db='../db/meth_db')


It took 0 s


In [29]:
#Create index for the database
# this one really takes long

run_sqlilte3("""
.separator "\t"

--index to fetch results fast from one sample
CREATE INDEX sample_probe_idx ON meth(sample,probe,beta);

--index to fetch result fast from a probe 
CREATE INDEX probe_sample_idx ON meth(probe,sample,beta);

""",db='../db/meth_db')


It took 729 s


#### Run  an example aggregating query

In [31]:
run_sqlilte3("""
.separator "\t"

EXPLAIN QUERY PLAN SELECT d.disease,AVG(av.abeta)
FROM(
    SELECT sample,AVG(beta) AS abeta
    FROM meth
    GROUP BY sample) as av
INNER JOIN disease_annot AS d ON av.sample=d.sample
GROUP BY d.disease;

SELECT d.disease,AVG(av.abeta)
FROM(
    SELECT sample,AVG(beta) AS abeta
    FROM meth
    GROUP BY sample) as av
INNER JOIN disease_annot AS d ON av.sample=d.sample
GROUP BY d.disease;

""",db='../db/meth_db')

1	0	0	SCAN TABLE meth USING COVERING INDEX sample_probe_idx
0	0	1	SCAN TABLE disease_annot AS d
0	1	0	SEARCH SUBQUERY 1 AS av USING AUTOMATIC COVERING INDEX (sample=?)
0	0	0	USE TEMP B-TREE FOR GROUP BY
adenoma	0.525580340649512
cancer	0.556780503734447
normal-C	0.559670418588082
normal-H	0.550449178010377

It took 33 s
